## Definition de la fonction

In [10]:
import pandas as pd

def add_metrics(data_lol_1):
    summoner_name_1 = data_lol_1["summonerName"][0]
    data_lol_1 = data_lol_1[data_lol_1["gameDuration"] > 300].copy()

    data_lol_1.loc[:, "firstMetric"] = (data_lol_1["kills"] + data_lol_1["assists"]) / (data_lol_1["ennemyTeamKills"] + data_lol_1["kills"] + data_lol_1["assists"])

    data_lol_1.loc[:, "secondMetric"] = data_lol_1["deaths"] / data_lol_1["ennemyTeamDeaths"]

    data_lol_1.loc[:, "thirdMetric"] = data_lol_1["totalDamageDealtToChampions"] / data_lol_1["ennemyTeamDamages"]

    data_lol_1.loc[:, "fourthMetric"] = data_lol_1["damageTakenOnTeamPercentage"]

    data_lol_1.loc[:, "fifthMetric"] = data_lol_1["goldPerMinute"] * data_lol_1["gameDuration"] / 60 / data_lol_1["ennemyTeamGolds"]

    data_lol_1.loc[:, "sixthMetric"] = data_lol_1["totalHeal"] / data_lol_1["ennemyTeamDamages"]

    # playerDamageMitigated / ennemyTeamDamages

    data_lol_1.loc[:, "eighthMetric"] = (data_lol_1["neutralMinionsKilled"] + data_lol_1["totalMinionsKilled"]) / data_lol_1["ennemyTeamGolds"] * 100

    data_lol_1.loc[:, "ninthMetric"] = data_lol_1["wardsPlaced"] / data_lol_1["ennemyTeamWardPlaced"]

    data_lol_1.loc[:, "tenthMetric"] = data_lol_1["wardTakedowns"] / data_lol_1["ennemyTeamWardPlaced"]

    data_lol_1.loc[:, "eleventhMetric"] = data_lol_1["champExperience"] / data_lol_1["ennemyTeamTotalLvl"]

    data_lol_1.loc[:, "twelvthMetric"] = data_lol_1["timeCCingOthers"] / data_lol_1["ennemyTeamDeaths"]

    return data_lol_1

## Importation du dataset

In [11]:
import os
import pandas as pd

# Chemin absolu du dossier courant
chemin_scouting = os.path.abspath('..')

print("Chemin du dossier parent :", chemin_scouting)

chemin_data = chemin_scouting + "/datas/matches/equipes/KC/lol_df_KC NEXT ADKING.pkl"

data_lol = pd.read_pickle(chemin_data) # dataset

data_lol = add_metrics(data_lol)
print(data_lol.shape)

Chemin du dossier parent : /Users/atysp/Desktop/Avisia/projet_lol/scouting
(100, 86)


## Calcul des coefficients de corrélation pour chaque variable

In [12]:
import pandas as pd
from scipy.stats import pointbiserialr

# List of required columns including "win"
required_columns = ["firstMetric", "secondMetric", "thirdMetric", "fourthMetric", "fifthMetric", "sixthMetric", "eighthMetric", "ninthMetric", "tenthMetric", "twelvthMetric", "win"]

# Check if all required columns exist in the DataFrame
if not all(col in data_lol.columns for col in required_columns):
    raise ValueError("Certaines colonnes ou la variable 'win' ne sont pas présentes dans le DataFrame.")

# Select the required columns in the DataFrame
data_lol = data_lol[required_columns]

correlation_coefficients = {}
for col in data_lol.columns[:-1]:
    x = data_lol[col]
    y = data_lol["win"]
    correlation_coefficients[col] = pointbiserialr(x, y)

print("Corrélation de point-biserial :", correlation_coefficients)
for key in correlation_coefficients.keys():
    print(key)
    print(correlation_coefficients[key])

    print()

Corrélation de point-biserial : {'firstMetric': SignificanceResult(statistic=0.4745462885786957, pvalue=6.102909280537261e-07), 'secondMetric': SignificanceResult(statistic=-0.35413745343019043, pvalue=0.0003006566366710498), 'thirdMetric': SignificanceResult(statistic=0.29349074884270443, pvalue=0.0030409930729756586), 'fourthMetric': SignificanceResult(statistic=-0.02400417164300838, pvalue=0.8126118375649606), 'fifthMetric': SignificanceResult(statistic=0.4504816558066639, pvalue=2.56510033464777e-06), 'sixthMetric': SignificanceResult(statistic=0.08445508788339293, pvalue=0.4034774751330919), 'eighthMetric': SignificanceResult(statistic=0.07313185551292381, pvalue=0.4696226730125897), 'ninthMetric': SignificanceResult(statistic=0.2877019476838458, pvalue=0.0037022134193212494), 'tenthMetric': SignificanceResult(statistic=0.016379295123934594, pvalue=0.8715070582591674), 'twelvthMetric': SignificanceResult(statistic=0.16049602512904532, pvalue=0.11068069043309267)}
firstMetric
Signi


Corrélation de point-biserial : 
Le coefficient de corrélation de point-biserial est une mesure de la force et de la direction de la corrélation linéaire entre les deux variables. Dans ce cas, le coefficient est d'environ 0.735. Comme il est positif, cela indique une corrélation positive entre les variables "x" et "y". Une corrélation positive signifie que lorsque les valeurs de "x" augmentent, les valeurs de "y" ont tendance à augmenter également. Plus le coefficient est proche de 1 (ou -1 pour une corrélation négative), plus la corrélation est forte.

P-valeur : 
La p-valeur est une mesure de la significativité statistique de la corrélation. En général, on utilise un seuil de signification de 0,05 (5 %) pour déterminer si la corrélation est statistiquement significative. Si la p-valeur est inférieure à 0,05, on considère que la corrélation est statistiquement significative.

##  Calcul du score par match

In [13]:
data_lol_standardized = data_lol[required_columns[:-1]]

means = data_lol_standardized.mean()
stds = data_lol_standardized.std()

# Standardisation des colonnes
data_lol_standardized = (data_lol_standardized - means) / stds

# Remplacement des colonnes d'origine par les colonnes standardisées dans le DataFrame "data"
data_lol[required_columns[:-1]] = data_lol_standardized

In [14]:
for key in correlation_coefficients.keys():
    data_lol_standardized[key]==data_lol_standardized[key] * correlation_coefficients[key][0]

In [15]:
sum = 0 
for key in correlation_coefficients.keys():
    sum+= correlation_coefficients[key][0]
sum

1.4625412794880075

In [16]:
data_lol_standardized[key] / sum
data_lol_standardized

,firstMetric,secondMetric,thirdMetric,fourthMetric,fifthMetric,sixthMetric,eighthMetric,ninthMetric,tenthMetric,twelvthMetric
0,-1.668365,0.685954,-1.733226,-1.193758,-1.046516,-0.50729,-0.409938,-0.20952,-0.986216,-0.658175
1,-0.452041,-0.48856,-1.035563,-2.007389,-0.559449,-0.729324,0.208588,-0.414811,-0.217012,1.129156
2,-0.925056,-0.249307,-1.038519,-0.10103,-0.124819,-0.539146,1.305161,0.547703,0.110845,-0.770536
3,-0.085837,-0.249307,0.488869,-0.170569,0.901001,0.601117,0.27323,0.157072,1.601106,-0.3629
4,-0.512107,1.551385,0.109918,1.279753,-0.590297,-0.40686,-0.041128,0.157072,-0.114975,-0.168021
...,...,...,...,...,...,...,...,...,...,...
95,-0.88966,1.972326,-0.493883,0.553417,-1.165947,-0.538608,-0.617364,-0.919053,-0.608704,0.195175
96,0.448939,-1.271569,0.355483,0.648124,1.032959,2.278499,1.162399,-0.457856,0.082516,-0.564105
97,-0.013451,0.212027,0.600477,0.697743,0.148297,0.143937,-0.547238,-1.544962,0.038081,-0.489839
98,0.148612,-0.335362,0.492495,-0.512619,-0.343087,-0.515796,0.002583,0.890256,-0.821387,-0.727705


In [17]:
data= data_lol_standardized.sum(axis=1)

0    -7.727049
1    -4.566405
2    -1.784704
3     3.153784
4     1.264741
        ...   
95   -2.512301
96    3.715388
97   -0.754928
98   -1.722009
99   -2.167923
Length: 100, dtype: float64

In [19]:
data.max()

11.852712200142655

In [20]:
data.min()

-7.792004482892481